# sos:
- Can not use Vstack or Hstack invlist objects, they are read only they can not be copied, you can not assign stuff to them etc etc..
- thus this code now is wrong, can't use it...

## I think:
- FAISS does not support resizing an inverted list to be larger. I don't know how to go from nlist = 10, to nlist = 11 ( crack one point) without doing .add_preassigned() again from scratch (which is not what we want...)

How do I want the prototype (python) implementation to be?
- start w/ knn()
- cracking will slow down brute force search
    - implemented as vstack invlists and need re-assignments
- refine:
    - inner kmeans, 
    - get points visited/refine is the most expensive operation ( has copy O(pts_copied) not O(invlists copied))
- reorg:
    - I think this is not as expensive


In the python prototype, the logic of the algorithm should be correct and we should be able to see if the general idea holds. But there are 0 optimizations and cracking is slow.


Todo:
- need to think of a way to estimate cost of .add() for 1 or multiple centroids and depending if it's global/local and how many points etc etc....
- eg. cracking by adding 100 new centroids vs just 1 is just as expensive for 1M points, so obviously one is a better choice etc.

Batching:
- if you batch it, it is much faster than if you go 1 query at a time
IVFFlat
    - ~20ms per query -> ~200 sec for 10k queries 3:20 secs: --> but if you batch them all together ~32 secs @16 threads
- batch also helps, but not as huge of a difference as in Brute Force

In [10]:
import os
import numpy as np
import logging
import faiss
from vasili_helpers import *
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# InteractiveShell.ast_node_interactivity = "last"

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
xb, xq, xt, gt, metric = load_sift1M(f"/pub/scratch/vmageirakos/datasets/ann-fvecs/sift-128-euclidean")
nb, d = xb.shape

Loading sift1M...done


dataset shape:
xb.shape=(1000000, 128)
gt.shape=(10000, 100)
xq.shape=(10000, 128)


##### Test cracking a larger index...

In [26]:
from vasili_helpers import *

index_to_crack, train_time, add_time = train_ivfflat(
    xb,
    nlist=1000,
    km_n_iter=10,
    km_max_pts=256,
    seed=1,
    store_dir=None,  # if you want to store the index
    verbose=True,
    metric=metric,
    store=False,
)

index_to_crack.invlists.nlist

# index_to_crack.own_invlists = False # ????

Kmeans... nlist=1000 km_n_iter=10 km_max_pts=256 seed=1 nredo=1
Training level-1 quantizer
Training level-1 quantizer on 1000000 vectors in 128D
Training IVF residual
IndexIVF: no residual training
IndexIVFFlat::add_core: added 1000000 / 1000000 vectors
	---> Index Train Time = 1012.9538299952401 ms | Add Time = 385.0520280029741 ms <---


1000

In [31]:
dir(index_to_crack.invlists)

['INVALID_CODE_SIZE',
 'SUBSET_TYPE_ELEMENT_RANGE',
 'SUBSET_TYPE_ID_MOD',
 'SUBSET_TYPE_ID_RANGE',
 'SUBSET_TYPE_INVLIST',
 'SUBSET_TYPE_INVLIST_FRACTION',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__swig_destroy__',
 '__weakref__',
 'add_entries',
 'add_entry',
 'code_size',
 'compute_ntotal',
 'copy_subset_to',
 'get_codes',
 'get_ids',
 'get_iterator',
 'get_single_code',
 'get_single_id',
 'imbalance_factor',
 'is_empty',
 'list_size',
 'merge_from',
 'nlist',
 'prefetch_lists',
 'print_stats',
 'release_codes',
 'release_ids',
 'reset',
 'resize',
 'this',
 'thisown',
 'update_entries',
 'update_entry',
 'use_iterator']

In [50]:
index_to_crack.invlists
faiss.downcast_InvertedLists(index_to_crack.invlists)

<faiss.swigfaiss_avx2.InvertedLists; proxy of <Swig Object of type 'faiss::InvertedLists *' at 0x7f3da1f85a70> >

<faiss.swigfaiss_avx2.ArrayInvertedLists; proxy of <Swig Object of type 'faiss::ArrayInvertedLists *' at 0x7f3da1e1aeb0> >

In [25]:
from vasili_helpers import *

index_to_crack, train_time, add_time = train_ivfflat(
    xb,
    nlist=1000,
    km_n_iter=10,
    km_max_pts=256,
    seed=1,
    store_dir=None,  # if you want to store the index
    verbose=True,
    metric=metric,
    store=False,
)

index_to_crack
index_to_crack.own_invlists = False # ????

blah = crack_single(
    index_to_crack=index_to_crack, 
    crack_loc=xq[0:1, :], 
    assignments=None, 
    metric=metric
    )

index_to_crack.nlist
index_to_crack.invlists.nlist
index_to_crack.nprobe=1
# index_to_crack.search(xq[0:1,:], k=10)

print(f"5) {index_to_crack.invlists.nlist=}")    
print(f"5) {blah.invlists.nlist=}")    
print(f"5) {blah=}")    

Kmeans... nlist=1000 km_n_iter=10 km_max_pts=256 seed=1 nredo=1
Training level-1 quantizer
Training level-1 quantizer on 1000000 vectors in 128D
Training IVF residual
IndexIVF: no residual training
IndexIVFFlat::add_core: added 1000000 / 1000000 vectors
	---> Index Train Time = 965.3408289959771 ms | Add Time = 387.23051799752284 ms <---


<faiss.swigfaiss_avx2.IndexIVFFlat; proxy of <Swig Object of type 'faiss::IndexIVFFlat *' at 0x7f3da1f7ec70> >

--- combined_invlists=<faiss.swigfaiss_avx2.VStackInvertedLists; proxy of <Swig Object of type 'faiss::VStackInvertedLists *' at 0x7f3da1f7f0c0> > ---
--- combined_invlists=<faiss.swigfaiss_avx2.VStackInvertedLists; proxy of <Swig Object of type 'faiss::VStackInvertedLists *' at 0x7f3da1f08de0> > ---

1) combined_index.invlists.nlist=1001


WARNING clustering 1 points to 1 centroids: please provide at least 39 training points


RuntimeError: Error in void faiss::IndexIVF::replace_invlists(faiss::InvertedLists*, bool) at /home/runner/miniconda3/conda-bld/faiss-pkg_1728491153420/work/faiss/IndexIVF.cpp:1236: Error: 'il->nlist == nlist' failed

In [13]:
index_to_crack.search(xq[0:1, :], 10)
index_to_crack.d
index_to_crack.search(xq[0:1, :], 10)

(array([[55091., 59531., 69844., 73793., 74016., 75124., 75554., 78092.,
         78889., 79508.]], dtype=float32),
 array([[934876, 561813, 701258, 600499, 893601, 562167, 746931, 779712,
          49874, 701919]]))

128

(array([[55091., 59531., 69844., 73793., 74016., 75124., 75554., 78092.,
         78889., 79508.]], dtype=float32),
 array([[934876, 561813, 701258, 600499, 893601, 562167, 746931, 779712,
          49874, 701919]]))

In [17]:
index_to_crackindex_to_crack.invlists.list_size(0)

: 

In [14]:
index_to_crack
index_to_crack.nlist

<faiss.swigfaiss_avx2.IndexIVFFlat; proxy of <Swig Object of type 'faiss::IndexIVFFlat *' at 0x7fcf83a3cae0> >

1001

# Crack


We create an index with centroid being the query

We need to create an Inverted Lists object with a single inverted list being the new centroid we wish to grow from

Let's start the index with a single inverted list, defined by centroid being the first point in the dataset, and from then on you can "crack" based on the queries

#### Q: is there a faster way to do the .add()? just add xb into the 1st invlist, instead of doing distance computations to .add() which I think is what's happening here:

In [ ]:
faiss.omp_set_num_threads(127)

(1000000, 128)

NOTE:
- add_preassigned I think calls `void IndexIVF::add_core` which parallelizes over inv lists and loops over points. ----> So O(#points) complexity in the .add() 
- 

In [257]:
# first_point = xb[0:1, :]
# quantizer = faiss.IndexFlatL2(d)
# TEST_idx = faiss.IndexIVFFlat(quantizer, d, 1, faiss.METRIC_L2)

# strain = time.perf_counter()
# TEST_idx.train(first_point)
# etrain = time.perf_counter()

# sadd = time.perf_counter()
# # crack_idx.add(xb) # this has some cost of a few queries through brute force
# eadd = time.perf_counter()

# TEST_idx.ntotal
# TEST_idx.nlist
# TEST_idx.invlists.imbalance_factor()
# TEST_idx.invlists.print_stats()
# TEST_idx.invlists.list_size(0)
# TEST_idx.is_trained
# print(f"\t---> Index Train Time = {(etrain - strain)*1000} ms | Add Time = {(eadd - sadd)*1000} ms <---")

# # ## add preassigned
# assignements = np.zeros(xb.shape[0], dtype='int64')
# from faiss.contrib.ivf_tools import add_preassigned
# # # you need to set this to
# # TEST_idx.is_trained = True

# start = time.perf_counter()
# add_preassigned(index_ivf=TEST_idx, x=xb, a=assignements)#, ids=xb_ids) # ids implied by order of pts
# end = time.perf_counter()

# TEST_idx.ntotal
# TEST_idx.nlist
# TEST_idx.invlists.imbalance_factor()
# TEST_idx.invlists.print_stats()
# TEST_idx.invlists.list_size(0)

# # it still takes ~600ms to add to invlist, so it's quite slow
# # adding to 1 invlist or ~100 for sift is about the same cost... <<< how to model this & why
# print(f"\t---> Add preassigned = {(end - start)*1000} ms <<<")

Q: should I try to add entries to add batch in inv list?
``` cpp
size_t ArrayInvertedLists::add_entries(
        size_t list_no,
        size_t n_entry,
        const idx_t* ids_in,
        const uint8_t* code) {
```

In [36]:
# # brute force serach specific invlist 
# from faiss.contrib.ivf_tools import search_preassigned
# TEST_idx.nprobe = 1
# search_preassigned(TEST_idx, xq, k=100, list_nos=np.zeros(xq.shape[0], dtype='int64').reshape(-1,1))

#### Crack (default/naive)

index_to_crack

In [11]:
first_point = xb[0:1, :]
quantizer = faiss.IndexFlatL2(d)
if metric == "euclidean":
    index_to_crack = faiss.IndexIVFFlat(quantizer, d, 1, faiss.METRIC_L2)
elif metric == "angular":
    index_to_crack = faiss.IndexIVFFlat(quantizer, d, 1, faiss.METRIC_INNER_PRODUCT)


strain = time.perf_counter()
index_to_crack.train(first_point)
etrain = time.perf_counter()

sadd = time.perf_counter()
index_to_crack.add(xb) # this has some cost of a few queries through brute force
eadd = time.perf_counter()

index_to_crack.ntotal
index_to_crack.nlist
index_to_crack.invlists.imbalance_factor()
index_to_crack.invlists.print_stats()
index_to_crack.invlists.list_size(0)
index_to_crack

print(f"\t---> Index Train Time = {(etrain - strain)*1000} ms | Add Time = {(eadd - sadd)*1000} ms <---")

: 

In [ ]:
from vasili_helpers import *

index_to_crack = crack_single(
    index_to_crack=index_to_crack, 
    crack_loc=xq[0:1, :], 
    assignments=None, 
    metric=metric
    )

- After crack, index_to_crack.nlist=2


WARNING clustering 1 points to 1 centroids: please provide at least 39 training points


From then on, for incoming queries you can decide to "crack" which means adding more centroids and corresponding invlists (partitions) to the index_to_crack
- if it's the very first one, we have to add all points to the same invlist, as we did above

In [6]:
crack_query = xq[0:1, :]
quantizer = faiss.IndexFlatL2(d)
if metric == "euclidean":
    new_crack = faiss.IndexIVFFlat(quantizer, d, 1, faiss.METRIC_L2)
elif metric == "angular":
    new_crack = faiss.IndexIVFFlat(quantizer, d, 1, faiss.METRIC_INNER_PRODUCT)


strain = time.perf_counter()
new_crack.train(crack_query)
etrain = time.perf_counter()

# NOTE: if you know the assignments at this point, you can add the points here to the invlist
#       - but make sure you've removed them from the other invlists (no copies)
sadd = time.perf_counter()
# new_crack.add()
eadd = time.perf_counter()

new_crack.ntotal
new_crack.nlist

print(f"\t---> Index Train Time = {(etrain - strain)*1000} ms | Add Time = {(eadd - sadd)*1000} ms <---")

WARNING clustering 1 points to 1 centroids: please provide at least 39 training points


0

1

	---> Index Train Time = 0.08796999463811517 ms | Add Time = 0.01172000338556245 ms <---


Then we need to merge the invlist of the running query to the cracked query

In [7]:
ilv = faiss.InvertedListsPtrVector() # a pointer for multiple invlists
# existing invlists
existing_invlists = index_to_crack.invlists # 
ilv.push_back(existing_invlists)
# adding the new one (crack)
new_invlist = new_crack.invlists
ilv.push_back(new_invlist)
combined_invlists = faiss.VStackInvertedLists(ilv.size(), ilv.data())
# big_il = faiss.HStackInvertedLists(ilv.size(), ilv.data())
combined_invlists.nlist

2

make sure that all points are still there

In [8]:
combined_invlists
ntotal_new = combined_invlists.compute_ntotal()
ntotal_new  # take care not to have query in xb of new index
assert ntotal_new == index_to_crack.ntotal

<faiss.swigfaiss_avx2.VStackInvertedLists; proxy of <Swig Object of type 'faiss::VStackInvertedLists *' at 0x7f72407aa6d0> >

1000000

Combine the centroids which at this point you already have no need to reconstruct, but as an exmple:

In [9]:
combined_centroids = np.vstack([index_to_crack.quantizer.reconstruct_n(), new_crack.quantizer.reconstruct_n()])
combined_centroids.shape

(2, 128)

In [10]:
combined_nlist = combined_invlists.nlist
combined_nlist

2

create a new empty index with the correct centroids and nlist size, to merge the above two. Don't think it's possible to merge in-place in index_to_crack, so we'll replace it later

In [12]:
%%time
combined_quantizer = faiss.IndexFlatL2(d)
combined_quantizer.add(combined_centroids)
# new empty index
combined_index = faiss.IndexIVFFlat(combined_quantizer, d, combined_nlist, faiss.METRIC_L2)

# combined_index.ntotal
# combined_index.is_trained
# combined_index.nlist
# combined_index.invlists.nlist
# combined_index.invlists.print_stats() # empty

CPU times: user 0 ns, sys: 82 μs, total: 82 μs
Wall time: 68.2 μs


TODO: I don' tknow if own_invlists should be True or False at this point, which is set in replace_invlists

In [13]:
%%time
combined_index.replace_invlists(combined_invlists, True) # True/False is wether own_invlists ( ie if index is deleted this arrayinvlists is deleted)

CPU times: user 25 μs, sys: 4 μs, total: 29 μs
Wall time: 35.5 μs


In [14]:
# combined_index.own_invlists
# combined_index.ntotal
# combined_index.is_trained
# combined_index.nlist
# combined_index.invlists.nlist
# combined_index.invlists.print_stats() # empty
# combined_index.ntotal # why 0?
# combined_index.own_invlists # <<<<<

You need to manually set the ntotal because it's not tracked correctly

In [15]:
combined_index.ntotal

0

In [16]:
combined_index.ntotal = combined_invlists.compute_ntotal()
combined_index.ntotal

1000000

The way we created it now, all points are in one invlist, and the other is empty

In [14]:
combined_index.invlists.list_size(0) # original one
combined_index.invlists.list_size(1) # cracked one (we didn't add any)

1000000

0

So you obviously also need to handle the assignments

If you query nprobe = 1 you should get recall < 1   
if you query nprobe = 2 you should get recall = 2 ( because like above it's brute force )

In [15]:
combined_index.nprobe = 1
a, b = combined_index.search(xq, k=100)
gt
compute_recall(b, gt, k=100)

array([[932085, 934876, 561813, ..., 398306, 931721, 989762],
       [413247, 413071, 706838, ..., 855176, 846198, 987074],
       [669835, 408764, 408462, ..., 310475, 971815, 937903],
       ...,
       [123855, 123351, 534149, ...,  90175, 685486, 416474],
       [755327, 755323, 840765, ..., 595134, 601257, 172180],
       [874343, 464509, 413340, ..., 360985, 419949, 223427]], dtype=int32)

0.510959

In [16]:
combined_index.nprobe = 2
a, b = combined_index.search(xq, k=100)
gt
compute_recall(b, gt, k=100)

array([[932085, 934876, 561813, ..., 398306, 931721, 989762],
       [413247, 413071, 706838, ..., 855176, 846198, 987074],
       [669835, 408764, 408462, ..., 310475, 971815, 937903],
       ...,
       [123855, 123351, 534149, ...,  90175, 685486, 416474],
       [755327, 755323, 840765, ..., 595134, 601257, 172180],
       [874343, 464509, 413340, ..., 360985, 419949, 223427]], dtype=int32)

0.999889

How much slower is this than brute force?
- invlist scanner is much slower than brute force in current faiss implementation
    - there's the overhead for finding nprobe nearest centroids, but still it's not fast
    - in theory we can speed it up greatly but not an issue for now

In [17]:
faiss.omp_set_num_threads(1)

In [18]:
num_queries = 100

In [19]:
%%time
index_to_crack.nprobe = 1
# a, b = index_to_crack.search(xq, k=100)
a, b = index_to_crack.search(xq[:num_queries,:], k=100)
compute_recall(b, gt, k=100)

CPU times: user 3.93 s, sys: 3.82 ms, total: 3.94 s
Wall time: 3.94 s


1.0

In [20]:
%%time
combined_index.nprobe = 2
# a, b = combined_index.search(xq, k=100)
a, b = combined_index.search(xq[:num_queries,:], k=100)
compute_recall(b, gt, k=100)

CPU times: user 3.18 s, sys: 71 μs, total: 3.18 s
Wall time: 3.18 s


1.0

In [21]:
%%time
from faiss.contrib.exhaustive_search import knn
# _, _ = knn(xq, xb, 100)
_, _ = knn(xq[:num_queries,:], xb, 100)

CPU times: user 385 ms, sys: 11.7 ms, total: 397 ms
Wall time: 398 ms


Now we need to update, such that index_to_crack is the new combined_idx and we don't take extra space...
- will the invlists be deleted because own_invlists=true?

In [22]:
# we have 3 indexes at this point, we should only move forward with index_to_crack
index_to_crack
new_crack
combined_index

<faiss.swigfaiss_avx2.IndexIVFFlat; proxy of <Swig Object of type 'faiss::IndexIVFFlat *' at 0x7fc4ee9a7c30> >

<faiss.swigfaiss_avx2.IndexIVFFlat; proxy of <Swig Object of type 'faiss::IndexIVFFlat *' at 0x7fc946c17f60> >

<faiss.swigfaiss_avx2.IndexIVFFlat; proxy of <Swig Object of type 'faiss::IndexIVFFlat *' at 0x7fc4ee7f9da0> >

In [23]:
del index_to_crack
index_to_crack = combined_index
index_to_crack
new_crack
combined_index

<faiss.swigfaiss_avx2.IndexIVFFlat; proxy of <Swig Object of type 'faiss::IndexIVFFlat *' at 0x7fc4ee7f9da0> >

<faiss.swigfaiss_avx2.IndexIVFFlat; proxy of <Swig Object of type 'faiss::IndexIVFFlat *' at 0x7fc946c17f60> >

<faiss.swigfaiss_avx2.IndexIVFFlat; proxy of <Swig Object of type 'faiss::IndexIVFFlat *' at 0x7fc4ee7f9da0> >

In [24]:
crack_idx.nlist

2

# Refine

# Reorg

# (skip) Compare .add() times for different num_clusters

In [80]:
faiss.omp_set_num_threads(1)

In [87]:
nlist = 1
n_iter = 0
max_pts = 256
seed = 42 
result_dir= None

index, train_time, add_time = train_ivfflat(
    xb,
    nlist=nlist,
    km_n_iter=n_iter,
    km_max_pts=max_pts,
    seed=seed,
    store_dir=None,  # if you want to store the index
    verbose=True,
    metric=metric,
)

Kmeans... nlist=1 km_n_iter=0 km_max_pts=256 seed=42 nredo=1
Training level-1 quantizer
Training level-1 quantizer on 1000000 vectors in 128D
Training IVF residual
IndexIVF: no residual training
	---> Index Train Time = 48.224473001027945 ms | Add Time = 392.23024599778 ms <---IndexIVFFlat::add_core: added 1000000 / 1000000 vectors



In [101]:
a, b = create_random_dataset_fast(d=128, nt=1, nb=10000000, nq=1, seed=1234)

dataset shape:
xb.shape=(10000000, 128)
xq.shape=(1, 128)


In [102]:
nlist = 1
n_iter = 0
max_pts = 256
seed = 42 
result_dir= None

index, train_time, add_time = train_ivfflat(
    a,
    nlist=nlist,
    km_n_iter=n_iter,
    km_max_pts=max_pts,
    seed=seed,
    store_dir=None,  # if you want to store the index
    verbose=True,
    metric=metric,
)

Kmeans... nlist=1 km_n_iter=0 km_max_pts=256 seed=42 nredo=1
Training level-1 quantizer
Training level-1 quantizer on 10000000 vectors in 128D
Training IVF residual
IndexIVF: no residual training
	---> Index Train Time = 539.1174030010006 ms | Add Time = 7559.182643002714 ms <---IndexIVFFlat::add_core: added 10000000 / 10000000 vectors



In [105]:
nlist = 1000
n_iter = 0
max_pts = 256
seed = 42 
result_dir= None

index, train_time, add_time = train_ivfflat(
    a,
    nlist=nlist,
    km_n_iter=n_iter,
    km_max_pts=max_pts,
    seed=seed,
    store_dir=None,  # if you want to store the index
    verbose=True,
    metric=metric,
)

Kmeans... nlist=1000 km_n_iter=0 km_max_pts=256 seed=42 nredo=1
Training level-1 quantizer
Training level-1 quantizer on 10000000 vectors in 128D
Training IVF residual
IndexIVF: no residual training
	---> Index Train Time = 600.1265039994905 ms | Add Time = 27258.726158997888 ms <---IndexIVFFlat::add_core: added 10000000 / 10000000 vectors



# (skip) Brute force, knn vs indexflat
- can just use knn() 
- or create IndexFlat() ( but has small add time )

In [25]:
# %%time
# bf_index = faiss.IndexFlatL2(d)
# bf_index.add(xb)

In [26]:
# %%time
# _, _ = bf_index.search(xq, k=100) # if you just run it what is the time showing vs what if I run it with my method...

In [27]:
# %%timeit
# bf_index.search(xq[0:1,:], k=100)

In [28]:
# %%time
# nq, d = xq.shape
# batch_size = 1
# latency_per_batch=[]
# topK = 100
# I = np.empty((nq, topK), dtype="int32")
# D = np.empty((nq, topK), dtype="float32")
# i0 = 0
# t00 = time.perf_counter()
# while i0 < nq:
#     t_batch_start = time.perf_counter()

#     if i0 + batch_size < nq:
#         i1 = i0 + batch_size
#     else:
#         i1 = nq
    
#     Di, Ii = bf_index.search(xq[i0:i1], topK)

#     I[i0:i1] = Ii
#     D[i0:i1] = Di

#     t_batch_end = time.perf_counter()
#     latency_per_batch.append(t_batch_end - t_batch_start)

#     i0 = i1 # prepare for next iter

# latency_ms_per_batch_this_run = np.array(latency_per_batch) * 1000
# if (
#     len(latency_ms_per_batch_this_run) > 2
# ):  # remove first and last batch latency
#     latency_ms_per_batch_this_run = latency_ms_per_batch_this_run[
#         1:-1
#     ]
# print(
#     "Median batch latency: {:.3f} ms".format(
#         np.median(latency_ms_per_batch_this_run)
#     ) + " | " +
#     "Mean batch latency: {:.3f} ms".format(
#         np.mean(latency_ms_per_batch_this_run)
#     ) + " | " +
#     "99th percentile batch latency: {:.3f} ms".format(
#         np.percentile(latency_ms_per_batch_this_run, 99)
#     )
# )
# print(f"Total time: {(time.perf_counter() - t00) *1000 } ms" )

In [29]:
# nlist = 1000
# n_iter = 0
# max_pts = 256
# seed =42 
# result_dir= None

# index, train_time, add_time = train_ivfflat(
#     xb,
#     nlist=nlist,
#     km_n_iter=n_iter,
#     km_max_pts=max_pts,
#     seed=seed,
#     store_dir=None,  # if you want to store the index
#     verbose=True,
#     metric=metric,
# )

In [30]:
# nq, d = xq.shape
# batch_size = 16
# latency_per_batch=[]
# topK = 100
# I = np.empty((nq, topK), dtype="int32")
# D = np.empty((nq, topK), dtype="float32")
# i0 = 0
# while i0 < nq:
#     t_batch_start = time.perf_counter()

#     if i0 + batch_size < nq:
#         i1 = i0 + batch_size
#     else:
#         i1 = nq
    
#     Di, Ii = index.search(xq[i0:i1], topK)

#     I[i0:i1] = Ii
#     D[i0:i1] = Di

#     t_batch_end = time.perf_counter()
#     latency_per_batch.append(t_batch_end - t_batch_start)

#     i0 = i1 # prepare for next iter

# latency_ms_per_batch_this_run = np.array(latency_per_batch) * 1000
# if (
#     len(latency_ms_per_batch_this_run) > 2
# ):  # remove first and last batch latency
#     latency_ms_per_batch_this_run = latency_ms_per_batch_this_run[
#         1:-1
#     ]
# print(
#     "Median batch latency: {:.3f} ms".format(
#         np.median(latency_ms_per_batch_this_run)
#     ) + " | " +
#     "Mean batch latency: {:.3f} ms".format(
#         np.mean(latency_ms_per_batch_this_run)
#     ) + " | " +
#     "99th percentile batch latency: {:.3f} ms".format(
#         np.percentile(latency_ms_per_batch_this_run, 99)
#     )
# )

In [31]:
# i0 = 0
# i1 = 1000

In [32]:
# faiss.omp_set_num_threads(1)

In [33]:
# %%time
# Di, Ii = bf_index.search(xq[0:1], topK)

In [34]:
# Di, Ii = bf_index.search(xq[0:i1], topK)
# i1

In [35]:
# %%time
# batch_size = 1
# i0 = 0
# while i0 < nq:
#     t_batch_start = time.perf_counter()

#     if i0 + batch_size < nq:
#         i1 = i0 + batch_size
#     else:
#         i1 = nq
    
#     Di, Ii = bf_index.search(xq[i0:i1], topK)

#     I[i0:i1] = Ii
#     D[i0:i1] = Di

#     t_batch_end = time.perf_counter()
#     latency_per_batch.append(t_batch_end - t_batch_start)

#     i0 = i1 # prepare for next iter

In [36]:
# from faiss.contrib.exhaustive_search import knn

In [37]:
# %time
# _, _ = knn(xq, xb, 100)

In [38]:
# %%timeit
# _, _ = bf_index.search(xq[199:200,:], 100)

In [39]:
# %%timeit
# _, _ = knn(xq[199:200,:], xb, 100)

In [40]:
# i0, i1

In [41]:
# %%time
# Di, Ii = index.search(xq[0:i1], topK)

In [42]:
# %%time
# batch_size = 1
# i0 = 0
# while i0 < nq:
#     t_batch_start = time.perf_counter()

#     if i0 + batch_size < nq:
#         i1 = i0 + batch_size
#     else:
#         i1 = nq
    
#     Di, Ii = index.search(xq[i0:i1], topK)

#     I[i0:i1] = Ii
#     D[i0:i1] = Di

#     t_batch_end = time.perf_counter()
#     latency_per_batch.append(t_batch_end - t_batch_start)

#     i0 = i1 # prepare for next iter


In [43]:
# 